In [1]:
import pathlib

import tensorflow as tf
import tensorflow_ranking as tfr
import tensorflow_text as tf_text
from tensorflow_serving.apis import input_pb2
from google.protobuf import text_format

2023-04-13 10:13:34.056047: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 10:13:34.749182: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/simin/miniconda3/envs/tf-rank/lib/:/home/simin/miniconda3/envs/tf-rank/lib/python3.10/site-packages/nvidia/cudnn/lib
2023-04-13 10:13:34.749260: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-13 10:13:39.831992: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 10:13:39.848197: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 10:13:39.848583: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
# Store the paths to files containing training and test instances.
_TRAIN_DATA_PATH = '../StackExchange/Stackoverflow/train1.tfrecords'
_TEST_DATA_PATH = '../StackExchange/Stackoverflow/test1.tfrecords'

# Store the vocabulary path for query and document tokens.
_VOCAB_PATH = '../StackExchange/Stackoverflow/vocab1.txt'

# The maximum number of documents per query in the dataset.
# Document lists are padded or truncated to this size.
_LIST_SIZE = 100

# The document relevance label.
_LABEL_FEATURE = "relevance"

# Padding labels are set negative so that the corresponding examples can be
# ignored in loss and metrics.
_PADDING_LABEL = -1

# Learning rate for optimizer.
_LEARNING_RATE = 0.05

# Parameters to the scoring function.
_BATCH_SIZE = 32
_HIDDEN_LAYER_DIMS = ["64", "32", "16"]
_DROPOUT_RATE = 0.8
_GROUP_SIZE = 1  # Pointwise scoring.

# Location of model directory and number of training steps.
_MODEL_DIR = "/tmp/ranking_model_dir2"
_NUM_TRAIN_STEPS = 80 * 1000
_EMBEDDING_DIMENSION = 20
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]","c++","c#","f#","node.js","nodejs",".json",".js",".net","objective-c",
                                  "asp.net","ruby-on-rails","angular.js","angular-js"]

In [4]:
def context_feature_columns():
    """Returns context feature names to column definitions."""
    sparse_column = tf.feature_column.categorical_column_with_vocabulary_file(
        key="query_tokens",
        vocabulary_file=_VOCAB_PATH)
    query_embedding_column = tf.feature_column.embedding_column(
        sparse_column, _EMBEDDING_DIMENSION)
    return {"query_tokens": query_embedding_column}


def example_feature_columns():
    """Returns the example feature columns."""
    sparse_column = tf.feature_column.categorical_column_with_vocabulary_file(
        key="document_tokens",
        vocabulary_file=_VOCAB_PATH)
    document_embedding_column = tf.feature_column.embedding_column(
        sparse_column, _EMBEDDING_DIMENSION)
    return {"document_tokens": document_embedding_column}

In [5]:
def input_fn(path, num_epochs=None):
    context_feature_spec = tf.feature_column.make_parse_example_spec(
        context_feature_columns().values())
    label_column = tf.feature_column.numeric_column(
        _LABEL_FEATURE, dtype=tf.int64, default_value=_PADDING_LABEL)
    example_feature_spec = tf.feature_column.make_parse_example_spec(
        list(example_feature_columns().values()) + [label_column])
    dataset = tfr.data.build_ranking_dataset(
        file_pattern=path,
        data_format=tfr.data.ELWC,
        batch_size=_BATCH_SIZE,
        list_size=_LIST_SIZE,
        context_feature_spec=context_feature_spec,
        example_feature_spec=example_feature_spec,
        reader=tf.data.TFRecordDataset,
        shuffle=False,
        num_epochs=num_epochs)
    features = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
    label = tf.squeeze(features.pop(_LABEL_FEATURE), axis=2)
    label = tf.cast(label, tf.float32)

    return features, label

In [6]:
def make_transform_fn():
    def _transform_fn(features, mode):
        """Defines transform_fn."""
        context_features, example_features = tfr.feature.encode_listwise_features(
            features=features,
            context_feature_columns=context_feature_columns(),
            example_feature_columns=example_feature_columns(),
            mode=mode,
            scope="transform_layer")

        return context_features, example_features
    return _transform_fn

In [7]:
def make_score_fn():
    """Returns a scoring function to build `EstimatorSpec`."""

    def _score_fn(context_features, group_features, mode, params, config):
        """Defines the network to score a group of documents."""
        with tf.compat.v1.name_scope("input_layer"):
            context_input = [
                tf.compat.v1.layers.flatten(context_features[name])
                for name in sorted(context_feature_columns())
            ]
            group_input = [
                tf.compat.v1.layers.flatten(group_features[name])
                for name in sorted(example_feature_columns())
            ]
            input_layer = tf.concat(context_input + group_input, 1)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)
        cur_layer = input_layer
        cur_layer = tf.compat.v1.layers.batch_normalization(
            cur_layer,
            training=is_training,
            momentum=0.99)

        for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
            cur_layer = tf.compat.v1.layers.dense(cur_layer, units=layer_width)
            cur_layer = tf.compat.v1.layers.batch_normalization(
                cur_layer,
                training=is_training,
                momentum=0.99)
            cur_layer = tf.nn.relu(cur_layer)
            cur_layer = tf.compat.v1.layers.dropout(
                inputs=cur_layer, rate=_DROPOUT_RATE, training=is_training)
        logits = tf.compat.v1.layers.dense(cur_layer, units=_GROUP_SIZE)
        return logits

    return _score_fn

In [8]:
def eval_metric_fns():
    """Returns a dict from name to metric functions.

    This can be customized as follows. Care must be taken when handling padded
    lists.

    def _auc(labels, predictions, features):
        is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
        clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
        clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
        return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
    metric_fns["auc"] = _auc

    Returns:
        A dict mapping from metric name to a metric function with above signature.
    """
    metric_fns = {}
   
    metric_fns.update({
        f"metric/ndcg@{topn}": tfr.metrics.make_ranking_metric_fn(
            tfr.metrics.RankingMetricKey.NDCG, topn=topn)
        for topn in [10,15,20,25,30]
    })
    """
    metric_fns.update({
        f"metric/map@{topn}": tfr.metrics.make_ranking_metric_fn(
            tfr.metrics.RankingMetricKey.MAP, topn=topn)
        for topn in [15, 20]
    })
    """
    
    return metric_fns

## Prepare dataset

### Create vocab.txt using bert

In [9]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import shutil
import string

In [10]:
train_path = "../StackExchange/Stackoverflow/train/"
df_train = pd.read_csv("../StackExchange/Stackoverflow/cluster_4d.csv")
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 16
seed = 42
class_list = sorted([str(folder) for folder in set(df_train["label"])])
num_class = len(class_list)

In [11]:
def remove_non_ascii(sentence):
    return ''.join(char for char in sentence if ord(char) < 128)

'''
def html_Filter(sentence):
    sentence = BeautifulSoup(sentence, "lxml").text
    #print("after html_Filter",sentence)
    
    return sentence
'''

def html_Filter(sentence):
    soup = BeautifulSoup(sentence, "html.parser")
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
    
    return ' '.join(soup.stripped_strings)



def keywords_transform(sentence):
    delimiter = [char for char in string.punctuation]
    punc_reserved = ['#','+','-',"'"]
    delimiter_filter = list(set(delimiter) - set(punc_reserved))
    
    sentence = sentence.lower()
    for char in delimiter_filter:
        sentence = sentence.replace(char,' '+char+' ')

    sentence = sentence.replace("node js","node.js")
    sentence = sentence.replace("node . js","node.js")
    sentence = sentence.replace(" . js",".js")
    sentence = sentence.replace(" . net",".net")
    sentence = sentence.replace("asp . net","asp.net")
    sentence = sentence.replace(" . json",".json")
    sentence = sentence.replace("objective c","objective-c")
    sentence = sentence.replace("ruby on rails","ruby-on-rails")
    sentence = sentence.replace("angular js","angular-js")
    sentence = sentence.replace("angular . js","angular.js")
    
        
    return sentence

In [12]:
def write_vocab_file(filepath, vocab):
    with open(filepath, 'w') as f:
        for token in vocab:
            print(token, file=f)

In [45]:
#transform dataset if needed
for name in class_list:
    newpath = train_path+name+'/' 
    if not os.path.exists(newpath):
        os.makedirs(newpath)

for i, sentence, tag, label in zip(df_train["Id"],df_train["sentence"],df_train["tag"],df_train["label"]):
    tag_string = tag.replace("<","")
    tag_string = tag_string.replace(">"," ")
    sentence += ' '+tag_string.strip()
    sentence = html_Filter(sentence)
    sentence = remove_non_ascii(sentence)
    sentence = keywords_transform(sentence)
    
    #vlabel_list.append(label)
    
    with open(train_path+str(label)+'/'+str(i)+".txt","w",encoding="UTF-8") as f:
        f.write(sentence)

In [46]:
#fetech dataset
train_ds = tf.keras.utils.text_dataset_from_directory(
    train_path,
    labels=None,
    label_mode = None,
    batch_size=batch_size,
    seed=seed)

Found 2194 files belonging to 1 classes.


In [53]:
bert_tokenizer_params=dict(lower_case=True)

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [54]:
en_vocab = bert_vocab.bert_vocab_from_dataset(
    train_ds.cache().prefetch(buffer_size=AUTOTUNE),
    **bert_vocab_args
)

In [55]:
print(len(en_vocab))

5964


In [56]:
print(en_vocab[:200])

['[PAD]', '[UNK]', '[START]', '[END]', 'c++', 'c#', 'f#', 'node.js', 'nodejs', '.json', '.js', '.net', 'objective-c', 'asp.net', 'ruby-on-rails', 'angular.js', 'angular-js', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', 'the', 'to', 'is', 'in', 'java', 'this', 'and', 'of', 'it', 'at', '##s', 'if', 'for', 'string', 'new', 'that', 'error', 'class', 'public', 'function', 'my', 'int', 'not', 'but', 'with', 'have', 'from', 'return', 'android', 'code', 'id', 'na', 'on', 'name', 'as', 'get', 'value', 'an', 'file', 'data', 'void', 'can', 'how', 'when', 'am', 'php', 'be', 'com', 'using', 'what', 'org', 'do', 'null', 'import', 'main', '##1', 'so', '##2', 'out', 'add', 'use', 'or', 'private', 'here', 'type'

In [57]:
write_vocab_file(_VOCAB_PATH, en_vocab)

### Create elwc dataset

In [13]:
'''
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    
    
    tokens = value.split()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[token.encode() for token in tokens]))
'''

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    
    
    value = html_Filter(value)
    value = remove_non_ascii(value)
    value = keywords_transform(value)
    whitespace_tokenizer = tf_text.WhitespaceTokenizer()
    words = whitespace_tokenizer.tokenize(value).numpy().tolist()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=words))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [14]:
def create_tfexample(feature0, feature1):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.
    feature = {
        'document_tokens': _bytes_feature(feature0),
        'relevance': _int64_feature(feature1), 
    }

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto

In [15]:
def create_tfquery(feature0):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.
    feature = {
        'query_tokens': _bytes_feature(feature0),
    }

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto

In [16]:
def parse_elwc(elwc,num_example):
    return tfr.data.parse_from_example_list(
        [elwc],
        list_size=num_example,
        context_feature_spec={"query_tokens": tf.io.RaggedFeature(dtype=tf.string)},
        example_feature_spec={
            "document_tokens":
                tf.io.RaggedFeature(dtype=tf.string),
            "relevance":
                tf.io.FixedLenFeature(shape=[], dtype=tf.int64, default_value=0)
        },
        size_feature_name="_list_size_",
        mask_feature_name="_mask_")

In [17]:
def create_serialized_elwc(tf_query, tf_examples):
    ELWC = input_pb2.ExampleListWithContext()
    ELWC.context.CopyFrom(tf_query)

    for example in tf_examples:
        example_features = ELWC.examples.add()
        example_features.CopyFrom(example)
    
    return ELWC.SerializeToString()

In [18]:
def create_dataset(dataset):
    ELWC_list = []
    
    for data in dataset:
        tag_string = data["tag"].replace("<","")
        tag_string = tag_string.replace(">"," ")
        query = data["query"] + ' '+tag_string.strip()
        tf_query = create_tfquery(query)
        
        EXAMPLES = []
        for doc in data["documents"]:
            tag_string = doc["tag"].replace("<","")
            tag_string = tag_string.replace(">"," ")
            example = doc["doc"] + ' '+tag_string.strip()
            tf_example = create_tfexample(example,doc["relevance"])
            EXAMPLES.append(tf_example)
        
        ELWC_list.append(create_serialized_elwc(tf_query, EXAMPLES))
    
    return ELWC_list

#### stackoverflow

In [64]:
import pandas as pd

In [65]:
df = pd.read_csv("../StackExchange/Stackoverflow/cluster_4d.csv")
df

,Id,sentence,tag,label
0,14191212,Alternative Function for EREGI in PHP <p>I was...,<php><regex><preg-match><eregi>,2867
1,14198874,JasperReports: Unsupported major.minor version...,<java><version><unsupported-class-version>,29595
2,1335851,"What does ""use strict"" do in JavaScript, and w...",<javascript><syntax><jslint><use-strict>,29590
3,1335886,mySQL Operand should contain 1 column error <p...,<mysql><sql><mysql-error-1241>,123659
4,1352885,Remove elements as you traverse a list in Pyth...,<python><list><loops><iterator><python-datamodel>,15259
...,...,...,...,...
2189,14113884,Protect C++ program against decompiling <block...,<c++><windows><obfuscation><decompiling><sourc...,29
2190,14115904,"$_POST not working. ""Notice: Undefined index: ...",<php><http-post><undefined-index>,8549
2191,14121385,Select Distinct Rows from MySQL database throu...,<php><mysql><pdo><haversine>,2802
2192,14123290,Pull data from PHP object received from twitte...,<php><twitter><twitter-streaming-api>,219989


In [66]:
label_set = set(df["label"])

In [67]:
len(label_set)

223

In [25]:
dataset = []

for label in label_set:
    df_relevant = df[df["label"] == label]    
    df_irrelevant = df[df["label"] != label]
    
    for index in list(df_relevant.index.values):
        data = {}
        data["qid"] = df_relevant.loc[index]["Id"]
        data["query"] = df_relevant.loc[index]["sentence"]
        data["label"] = df_relevant.loc[index]["label"]
        data["tag"] = df_relevant.loc[index]["tag"]
        
        
        docs = []
        df_doc = df_relevant[df_relevant["Id"] != df_relevant.loc[index]["Id"]]
        for index_d in list(df_doc.index.values):
            doc = {}
            doc["doc_id"] = df_doc.loc[index_d]["Id"]
            doc["doc"] = df_doc.loc[index_d]["sentence"]
            doc["tag"] = df_doc.loc[index_d]["tag"]
            doc["relevance"] = 1
            
            docs.append(doc)
        
        
        if len(docs) < _LIST_SIZE:
            num_irrelevant = _LIST_SIZE - len(docs)
            df_irdoc = df_irrelevant.sample(num_irrelevant)
            for index_d in list(df_irdoc.index.values):
                doc = {}
                doc["doc_id"] = df_irdoc.loc[index_d]["Id"]
                doc["doc"] = df_irdoc.loc[index_d]["sentence"]
                doc["tag"] = df_irdoc.loc[index_d]["tag"]
                doc["relevance"] = 0
            
                docs.append(doc)
        
        data["documents"] = docs
        
        dataset.append(data)

In [26]:
#train test split
train = []
test = []
has_label = set()

for data in dataset:
    if data["label"] in has_label:
        train.append(data)
    else:
        test.append(data)
        has_label.add(data["label"])

In [68]:
print("train:",len(train))
print("test:",len(test))

train: 1971
test: 223


In [69]:
elwc_train = create_dataset(train)

In [70]:
len(elwc_train)

1971

In [71]:
with tf.io.TFRecordWriter(_TRAIN_DATA_PATH) as writer:
    for objs in elwc_train:
        writer.write(objs)

In [72]:
elwc_test = create_dataset(test)

In [73]:
with tf.io.TFRecordWriter(_TEST_DATA_PATH) as writer:
    for objs in elwc_test:
        writer.write(objs)

## Losses, Metrics and Ranking Head

In [19]:
# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.

_LOSS = tfr.losses.RankingLossKey.APPROX_NDCG_LOSS
loss_fn = tfr.losses.make_loss_fn(_LOSS)

In [20]:
optimizer = tf.compat.v1.train.AdagradOptimizer(
    learning_rate=_LEARNING_RATE)


def _train_op_fn(loss):
    """Defines train op used in ranking head."""
    update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)
    minimize_op = optimizer.minimize(
        loss=loss, global_step=tf.compat.v1.train.get_global_step())
    train_op = tf.group([update_ops, minimize_op])
    return train_op


ranking_head = tfr.head.create_ranking_head(
      loss_fn=loss_fn,
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)

In [21]:
model_fn = tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(),
          transform_fn=make_transform_fn(),
          group_size=_GROUP_SIZE,
          ranking_head=ranking_head)

INFO:tensorflow:Building groupwise ranking model.


## Train and evaluate

In [22]:
def train_and_eval_fn():
    """Train and eval function used by `tf.estimator.train_and_evaluate`."""
    run_config = tf.estimator.RunConfig(
        save_checkpoints_steps=5000)
    ranker = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir=_MODEL_DIR,
        config=run_config)

    train_input_fn = lambda: input_fn(_TRAIN_DATA_PATH)
    eval_input_fn = lambda: input_fn(_TEST_DATA_PATH, num_epochs=1)

    train_spec = tf.estimator.TrainSpec(
        input_fn=train_input_fn, max_steps=_NUM_TRAIN_STEPS)
    eval_spec = tf.estimator.EvalSpec(
        name="eval",
        input_fn=eval_input_fn,
        throttle_secs=15)
    return (ranker, train_spec, eval_spec)

In [23]:
ranker, train_spec, eval_spec = train_and_eval_fn()

INFO:tensorflow:Using config: {'_model_dir': '/tmp/ranking_model_dir2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [24]:
! rm -rf "/tmp/ranking_model_dir2"  # Clean up the model directory.
#ranker, train_spec, eval_spec = train_and_eval_fn()
tf.estimator.train_and_evaluate(ranker, train_spec, eval_spec)

/bin/bash: /home/simin/miniconda3/envs/tf-rank/lib/libtinfo.so.6: no version information available (required by /bin/bash)
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 5000 or save_checkpoints_secs None.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:vocabulary_size = 5964 in query_tokens is inferred from the number of elements in the vocabulary_file ../StackExchange/Stackoverflow/vocab1.txt.
INFO:tensorflow:vocabulary_size = 5964 in document_tokens is inferred from the number of elements in the vocabulary_file ../StackExchange/Stackoverflow/vocab1.txt.
Instructions for updating:
Lambda fuctions will be no mo

/tmp/ipykernel_9557/2110212537.py:8: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  tf.compat.v1.layers.flatten(context_features[name])
/tmp/ipykernel_9557/2110212537.py:12: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  tf.compat.v1.layers.flatten(group_features[name])
/tmp/ipykernel_9557/2110212537.py:19: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  cur_layer = tf.compat.v1.layers.batch_normalization(
/tmp/ipykernel_9557/2110212537.py:25: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layer

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


2023-04-13 10:14:37.471545: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 10:14:37.472891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 10:14:37.473276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 10:14:37.473639: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_nod

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/ranking_model_dir2/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = -0.37449712, step = 0


...

2023-04-13 11:11:25.777821: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 11:11:25.778502: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 11:11:25.778977: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 11:11:25.780617: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 11:11:25.780656: I tensorflow/core/co

INFO:tensorflow:Inference Time : 1.20510s
INFO:tensorflow:Finished evaluation at 2023-04-13-11:11:26
INFO:tensorflow:Saving dict for global step 75000: global_step = 75000, labels_mean = 0.08838565, logits_mean = -1.7312726, loss = -0.63247824, metric/ndcg@10 = 0.43860665, metric/ndcg@15 = 0.48314726, metric/ndcg@20 = 0.5184135, metric/ndcg@25 = 0.54259485, metric/ndcg@30 = 0.5634837
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 75000: /tmp/ranking_model_dir2/model.ckpt-75000
INFO:tensorflow:global_step/sec: 14.5854
INFO:tensorflow:loss = -0.5737903, step = 75000 (6.856 sec)
INFO:tensorflow:global_step/sec: 22.0784
INFO:tensorflow:loss = -0.686081, step = 75100 (4.529 sec)
INFO:tensorflow:global_step/sec: 21.7362
INFO:tensorflow:loss = -0.9199672, step = 75200 (4.600 sec)
INFO:tensorflow:global_step/sec: 23.0976
INFO:tensorflow:loss = -0.6107001, step = 75300 (4.330 sec)
INFO:tensorflow:global_step/sec: 22.1103
INFO:tensorflow:loss = -0.6733981, step = 75400 (4.523 s

2023-04-13 11:15:14.294440: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 11:15:14.295083: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 11:15:14.295588: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 11:15:14.296349: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 11:15:14.296383: I tensorflow/core/co

INFO:tensorflow:Inference Time : 1.14025s
INFO:tensorflow:Finished evaluation at 2023-04-13-11:15:15
INFO:tensorflow:Saving dict for global step 80000: global_step = 80000, labels_mean = 0.08838565, logits_mean = -2.036921, loss = -0.6290036, metric/ndcg@10 = 0.4359496, metric/ndcg@15 = 0.4803662, metric/ndcg@20 = 0.5101482, metric/ndcg@25 = 0.53360116, metric/ndcg@30 = 0.55688655
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 80000: /tmp/ranking_model_dir2/model.ckpt-80000
INFO:tensorflow:Loss for final step: -0.56614244.


({'labels_mean': 0.08838565,
  'logits_mean': -2.036921,
  'loss': -0.6290036,
  'metric/ndcg@10': 0.4359496,
  'metric/ndcg@15': 0.4803662,
  'metric/ndcg@20': 0.5101482,
  'metric/ndcg@25': 0.53360116,
  'metric/ndcg@30': 0.55688655,
  'global_step': 80000},
 [])

## Inference and Predictions

We show how to generate predictions over the features of a dataset. We assume that the label is not present and needs to be inferred using the ranking model.

Similar to the `input_fn` used for training and evaluation,  `predict_input_fn` reads in data in ELWC format and stored as TFRecords to generate features. We set number of epochs to be 1, so that the generator stops iterating when it reaches the end of the dataset. Also the datapoints are not shuffled while reading, so that the behavior of the `predict()` function is deterministic.

In [25]:
def predict_input_fn(path):
    context_feature_spec = tf.feature_column.make_parse_example_spec(
        context_feature_columns().values())
    example_feature_spec = tf.feature_column.make_parse_example_spec(
        list(example_feature_columns().values()))
    dataset = tfr.data.build_ranking_dataset(
        file_pattern=path,
        data_format=tfr.data.ELWC,
        batch_size=_BATCH_SIZE,
        list_size=_LIST_SIZE,
        context_feature_spec=context_feature_spec,
        example_feature_spec=example_feature_spec,
        reader=tf.data.TFRecordDataset,
        shuffle=False,
        num_epochs=1)
    features = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
    return features

We generate predictions on the test dataset, where we only consider context and example features and predict the labels. The `predict_input_fn` generates predictions on a batch of datapoints. Batching allows us to iterate over large datasets which cannot be loaded in memory.

In [27]:
predictions = ranker.predict(input_fn=lambda: predict_input_fn(_TEST_DATA_PATH), checkpoint_path = ranker.latest_checkpoint())

`ranker.predict` returns a generator, which we can iterate over to create predictions, till the generator is exhausted.

In [28]:
x = next(predictions)

INFO:tensorflow:vocabulary_size = 5964 in query_tokens is inferred from the number of elements in the vocabulary_file ../StackExchange/Stackoverflow/vocab1.txt.
INFO:tensorflow:vocabulary_size = 5964 in document_tokens is inferred from the number of elements in the vocabulary_file ../StackExchange/Stackoverflow/vocab1.txt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:vocabulary_size = 5964 in query_tokens is inferred from the number of elements in the vocabulary_file ../StackExchange/Stackoverflow/vocab1.txt.
INFO:tensorflow:vocabulary_size = 5964 in document_tokens is inferred from the number of elements in the vocabulary_file ../StackExchange/Stackoverflow/vocab1.txt.
INFO:tensorflow:vocabulary_size = 5964 in query_tokens is inferred from the number of elements in the vocabulary_file ../StackExchange/Stackoverflow/vocab1.txt.
INFO:tensorflow:vocabulary_size = 5964 in document_tokens is inferred from the number of elements in the vocabulary_file ../StackExchange/Stackoverflow/voc

/tmp/ipykernel_9557/2110212537.py:8: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  tf.compat.v1.layers.flatten(context_features[name])
/tmp/ipykernel_9557/2110212537.py:12: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  tf.compat.v1.layers.flatten(group_features[name])
/tmp/ipykernel_9557/2110212537.py:19: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  cur_layer = tf.compat.v1.layers.batch_normalization(
/tmp/ipykernel_9557/2110212537.py:25: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layer

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/ranking_model_dir2/model.ckpt-80000


2023-04-13 23:31:46.367228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 23:31:46.368162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 23:31:46.368629: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 23:31:46.371856: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 23:31:46.371898: I tensorflow/core/co

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [29]:
x

array([ 0.11030643,  1.068628  , -1.193237  , -3.2113945 , -0.3502589 ,
       -1.2521886 , -0.60464126, -1.3437957 , -1.7907504 ,  0.7912551 ,
       -1.9312412 , -0.6163245 , -0.28500426, -2.2479465 ,  4.7539926 ,
       -2.3935404 , -1.7506343 , -2.5653894 , -1.2723231 , -2.1217813 ,
        0.41525897, 10.724654  ,  6.3025565 ,  2.6733677 , -1.2480651 ,
       -0.89325863,  0.21330483, -0.7000864 ,  0.16781536,  0.49158466,
       -4.091258  , -4.3734603 , -5.695457  ,  0.8357709 ,  0.7534142 ,
       -1.3700782 ,  0.57691455,  4.40552   , -3.9967525 , -0.6031495 ,
       -2.1061022 , -3.0913947 ,  5.2450986 ,  3.575774  , -7.1628213 ,
       -2.1439857 ,  1.8048462 , -8.92773   , -5.502502  ,  3.0449636 ,
       -3.171954  ,  7.69224   , -4.2807536 , -3.1045454 , -5.5898147 ,
       -4.8819    , -1.0271748 ,  0.7564909 , -0.75493735,  2.7342317 ,
       -0.06257825, -0.8753093 ,  4.357507  , -1.6211015 ,  0.87937874,
       -0.66134495, -0.67206615,  0.6673526 ,  3.612667  , -2.14

In [30]:
x2 = next(predictions)

In [31]:
x2

array([ 20.973263  ,  17.09673   ,  13.169425  ,  16.274303  ,
        17.291176  ,  13.715188  ,  -4.7571187 ,  -6.941793  ,
        -6.175723  ,  -5.5665503 ,  -2.7041383 ,  -6.376344  ,
        -2.2829974 ,  18.833288  ,  -5.716882  ,  -3.215854  ,
        -3.1922626 ,  -9.3092985 ,  -4.170464  ,  -3.6266415 ,
        -1.3036622 ,  -5.4949846 ,  -4.5339613 ,   2.0997164 ,
        -3.5297594 ,  -0.9621393 ,  -4.033933  ,   0.5803681 ,
        -3.3392985 ,  -1.3197008 ,  -1.5215089 ,  -1.937019  ,
        -4.7749863 ,  -4.3586135 ,  -3.7780015 ,  -6.1432805 ,
        23.08628   ,  -6.8669505 ,  -9.590476  ,  -1.8483611 ,
        -4.9096484 ,  -4.911651  ,  -4.0619025 ,   0.49486762,
        -2.7750752 ,  -3.5110867 , -10.211501  , -13.408258  ,
        -4.6502423 ,  -7.7602897 ,  -0.11777582,  -7.939951  ,
        -3.1290004 ,  15.996386  ,  -8.233302  ,  -3.7229564 ,
        -3.1207817 ,  -2.9017193 ,  -3.506057  ,  -2.0014176 ,
        -8.869835  ,  -0.76471597,  -2.8754084 ,  -2.97